# MP 2: Contact and Grasp Planning

**Due date**: Feb 21, 2022 at 9:45am.

**Instructions**: Read and complete the problems below. To submit your assignment, perform the following:

1. Double-check that your notebook runs without error from a clean "Restart and Clear Output".
2. Download your completed notebook.  To avoid sending unnecessarily large notebooks, choose "Restart and Clear Output" before downloading. 
3. Submit your notebook (and any other new files you created) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).

In [ ]:
#Imports

#If you have wurlitzer installed, this will help you catch printouts from Klamp't
#Note: doesn't work on Windows
#%load_ext wurlitzer

import time
from klampt import *
from klampt import vis
from klampt.math import vectorops,so3,se3
from klampt.model.trajectory import Trajectory
from klampt.io import numpy_convert
from klampt.model.contact import ContactPoint
import numpy as np
import math
import random
import os
import sys
sys.path.append('../common')
import known_grippers
vis.init('IPython')

closeup_viewport = {'up': {'z': 0, 'y': 1, 'x': 0}, 'target': {'z': 0, 'y': 0, 'x': 0}, 'near': 0.1, 'position': {'z': 1.0, 'y': 0.5, 'x': 0.0}, 'far': 1000}

## Problem 1: Reaching antipodal grasps

Problem 1 asks to find gripper transforms that match *antipodal grasps*.  An antipodal graph defines a line segment that a gripper could plausibly use for closing onto the geometry.  Here we will use the `AntipodalGrasp` data structure to represent such a grasp.  The key defining parameters of an antipodal grasp include the center of the line segment and its axis.  You should match this to the *closure axis* of the gripper, which is the direction in which the fingers will close down on the object.

The gripper itself is represented as a [GripperInfo](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.model.robotinfo.html#klampt.model.robotinfo.GripperInfo) data structure. For visualization purposes, we get the value `gripper_geom` which is a Klamp't [Geometry3D](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html#klampt.Geometry3D) instance.  You don't need to know the details of these objects for the purpose of completing this problem, but these details may be helpful in later problems.

In [ ]:
finger_radius = 0.01

class AntipodalGrasp:
    """A structure containing information about antipodal grasps.
    
    Attributes:
        center (3-vector): the center of the fingers (object coordinates).
        axis (3-vector): the direction of the line through the
            fingers (object coordinates).
        approach (3-vector, optional): the direction that the fingers
            should move forward to acquire the grasp.
        finger_width (float, optional): the width that the gripper should
            open between the fingers.
        contact1 (ContactPoint, optional): a point of contact on the
            object.
        contact2 (ContactPoint, optional): another point of contact on the
            object.
    """
    def __init__(self,center,axis):
        self.center = center
        self.axis = axis
        self.approach = None
        self.finger_width = None
        self.contact1 = None
        self.contact2 = None

    def add_to_vis(self,name,color=(1,0,0,1)):
        if self.finger_width == None:
            w = 0.05
        else:
            w = self.finger_width*0.5+finger_radius
        a = vectorops.madd(self.center,self.axis,w)
        b = vectorops.madd(self.center,self.axis,-w)
        vis.add(name,[a,b],color=color)
        if self.approach is not None:
            vis.add(name+"_approach",[self.center,vectorops.madd(self.center,self.approach,0.05)],color=(1,0.5,0,1))

#define some quantities of the gripper
gripper = known_grippers.robotiq_85
finger_tip = vectorops.madd(gripper.center,gripper.primaryAxis,gripper.fingerLength-0.005)
finger_closure_axis = gripper.secondaryAxis

temp_world = WorldModel()
res = temp_world.readFile(os.path.join('../data/gripperinfo',gripper.klamptModel))
if res == False:
    raise IOError("Unable to load file",gripper.klamptModel)
#merge the gripper parts into a static geometry
gripper_geom = gripper.getGeometry(temp_world.robot(0))

world2 = WorldModel()
obj2 = world2.makeRigidObject("object1")
obj2.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")

#this will perform a reasonable center of mass / inertia estimate
m = obj2.getMass()
m.estimate(obj2.geometry(),mass=0.908,surfaceFraction=0.0)
obj2.setMass(m)

#make the object transparent yellow
obj2.appearance().setColor(0.8,0.8,0.2,0.5)
world2.readFile("../data/terrains/plane.env")
world2.terrain(0).geometry().scale(0.1)
world2.terrain(0).appearance().setColor(0,0,0.5,0.5)

A. In the following cell you will create a function that takes an antipodal grasp as input, and then builds a rigid transform that matches `gripper_center` with the grasps's center and `gripper_closure_axis` with the grasp's axis.  The subsequent cell will show whether you've done this properly.

In [ ]:
########################## Problem 1.A code goes here ##############################

def match_grasp(finger_tip,finger_closure_axis,grasp):
    """
    Args:
        finger_tip (3-vector): local coordinates of the center-point between the gripper's fingers.
        finger_closure_axis (3-vector): local coordinates of the axis connecting the gripper's fingers.
        grasp (AntipodalGrasp): the desired grasp
        
    Returns:
        (R,t): a Klampt se3 element describing the maching gripper transform
    """
    return se3.identity()

####################################################################################

In [ ]:
#Problem 1.A. Find a rotation to match the gripper to the antipodal grasp

grasp1 = AntipodalGrasp([0.025,-0.15,0.015],[math.cos(math.radians(20)),math.sin(math.radians(20)),0])
grasp1.finger_width = 0.05
gripper_geom.setCurrentTransform(*match_grasp(finger_tip,finger_closure_axis,grasp1))

vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world2)
vis.add("gripper",gripper_geom)

grasps = [grasp1]
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    g.add_to_vis(name,(1,0,0,1)) 
vis.show()

B. You might have found that your gripper is posed while colliding with the table! This gripper pose is only one of many poses that meets the constraints of the antipodal grasp.  We will try to find a collision-free pose by random sampling and collision detection.  Implement the `match_grasp_sample` function that will sample uniformly from the set of possible gripper poses that meets the constraints of the antipodal grasp.  (You do not need to consider symmetry, although if your gripper was asymmetric a sampling function may wish to consider mirrored antipodal grasps)

When you run the cell after this and start clicking the buttons underneath the visualizer, you should see the gripper turn green to show that it's feasible, and red if it's not.  The first "Sample pose" button will just call your function once.  The second button will sample up to 100 samples, and return the first feasible one.  

In [ ]:
########################## Problem 1.B code goes here ##############################

def match_grasp_sample(gripper_center,gripper_closure_axis,grasp):
    """Sample a transform for the gripper that meets the grasp constraints.
    
    Args:
        gripper_center (3-vector): local coordinates of the center-point between the gripper's fingers.
        gripper_closure_axis (3-vector): local coordinates of the axis connecting the gripper's fingers.
        grasp (AntipodalGrasp): the desired grasp
    
    Returns:
        (R,t): a Klampt se3 element describing a randomly sampled maching gripper transform
    """
    R0,t0 = match_grasp(gripper_center,gripper_closure_axis,grasp)
    return (R0,t0)

####################################################################################

In [ ]:
#Problem 1.B. Find a collision-free pose that meets the grasp

def dosample():
    T = match_grasp_sample(finger_tip,finger_closure_axis,grasp1)
    gripper_geom.setCurrentTransform(*T)
    vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
    feasible = False
    if not gripper_geom.collides(world2.terrain(0).geometry()):
        if not gripper_geom.collides(obj2.geometry()):
            feasible = True
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
    
def dofeas():
    feasible = False
    for i in range(100):
        T = match_grasp_sample(finger_tip,finger_closure_axis,grasp1)
        gripper_geom.setCurrentTransform(*T)
        if not gripper_geom.collides(world2.terrain(0).geometry()):
            if not gripper_geom.collides(obj2.geometry()):
                #success!
                feasible = True
                vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
                break
    if not feasible:
        print("Unable to sample a feasible pose in 100 samples??")
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
        
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world2)
vis.add("gripper",gripper_geom)
vis.addAction(dosample,"Sample pose")
vis.addAction(dofeas,"Generate feasible pose")

grasps = [grasp1]
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    g.add_to_vis(name,(1,0,0,1)) 
vis.show()

C. Now, let's suppose that the given grasp is *object-centric*, i.e., we've stored it for the object in a database, and when we recognize the object's true pose, we can then recognize where to grasp it.  Implement the `object_centric_match` function to sample a transform for the gripper that meets the grasp constraints, for the object in its current transform.

In [ ]:
########################## Problem 1.C code goes here ##############################

def object_centric_match(finger_tip,finger_closure_axis,grasp_local,obj):
    """Sample a transform for the gripper that meets the desired grasp
    for a RigidObjectModel. 
    
    Args:
        finger_tip (3-vector): local coordinates of the center-point between the gripper's fingers.
        finger_closure_axis (3-vector): local coordinates of the axis connecting the gripper's fingers.
        grasp_local (AntipodalGrasp): the desired grasp, with coordinates given in the local frame of
            obj.
        obj (RigidObjectModel): the object to be grasped, posed in the world according to its
            estimated transform.
    
    Returns:
        (R,t): a Klampt se3 element describing a randomly sampled maching gripper transform.
    """
    #TODO: code goes here
    return match_grasp_sample(finger_tip,finger_closure_axis,grasp_local)

####################################################################################

In [ ]:
#Problem 1.C. Find a collision-free pose that meets the grasp

def do_randomize_object():
    if random.random() < 0.5:
        flipz = so3.rotation((1,0,0),math.pi)
    else:
        flipz = so3.identity()
    obj2.setTransform(so3.mul(flipz,so3.rotation((0,0,1),random.uniform(0,math.pi*2))),
                             [random.uniform(-0.4,0.4),random.uniform(-0.4,0.4),0.02])
    vis.update()

def do_feas_grasp():    
    feasible = False
    for i in range(100):
        T = object_centric_match(gripper_center,gripper_closure_axis,grasp1,obj2)
        gripper_geom.setCurrentTransform(*T)
        if not gripper_geom.collides(world2.terrain(0).geometry()):
            if not gripper_geom.collides(obj2.geometry()):
                #success!
                feasible = True
                vis.nativeWindow().setTransform("gripper",R=T[0],t=T[1])
                break
    if not feasible:
        print("Unable to sample a feasible pose in 100 samples??")
    if not feasible:
        vis.setColor("gripper",1,0,0)
    else:
        vis.setColor("gripper",0,1,0)
        
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world2)
vis.add("gripper",gripper_geom)
vis.addAction(do_randomize_object,"Randomize object")
vis.addAction(do_feas_grasp,"Generate feasible grasp")
vis.show()

## Problem 2: Generating simulation test worlds

In Problem 2 you will generate a small world with several objects on a flat table.  The `stable_faces` function provided will calculate a set of faces that the object can use to stably rest on a flat surface. 

A. In the `sample_arrangement()` function, sample k copies of the object so that they rest on the table with orientations chosen randomly from the stable faces.  The orientation about the stable face should also be chosen randomly.  You should test your function with some of the non-cube objects, such as those listed in the second cell.

Once you create a `RigidObjectModel` using `WorldModel.makeRigidObject()` and copy its geometry and mass, you should set its current transform using `RigidObjectModel.setTransform()`.  (Do not modify the geometry itself using `object.geometry().transform()`)

In [ ]:
from stable_faces import stable_faces  #in ../common

In [ ]:
#debugging the stable faces
world = WorldModel()
obj = world.makeRigidObject("object1")
#obj.geometry().loadFile("../data/objects/ycb-select/002_master_chef_can/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/003_cracker_box/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/011_banana/nontextured.ply"); 
#obj.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")
obj.geometry().loadFile("../data/objects/cube.off"); obj.geometry().scale(0.2)
#weird bug in Qhull -- cylinder can't be converted to ConvexHull
#obj.geometry().loadFile("../data/objects/cylinder.off")

#this will perform a reasonable center of mass / inertia estimate
m = obj.getMass()
m.estimate(obj.geometry(),mass=0.454,surfaceFraction=0.2)
obj.setMass(m)

#make the object transparent yellow
obj.appearance().setColor(0.8,0.8,0.2,0.5)

sides = stable_faces(obj,stability_tol=-0.01,merge_tol=0.05)
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)
for i,f in enumerate(sides):
    gf = GeometricPrimitive()
    gf.setPolygon(np.stack(f).flatten())
    color = (0.5+0.5*random.random(),0.5+0.5*random.random(),0.5+0.5*random.random(),1)
    vis.add("face{}".format(i),Geometry3D(gf),color=color)
vis.show()

In [ ]:
#parameters that will be useful for your code
table_width = 0.5
table_depth = 0.5
table_height = 0.3

from klampt.model.create import primitives
table_world = WorldModel()
table_surf = primitives.box(table_width,table_depth,0.05,center=(0,0,table_height-0.025),world=table_world,mass=float('inf'))
table_legs = []
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,-table_depth*0.5+0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(-table_width*0.5+0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_legs.append(primitives.box(0.05,0.05,table_height,center=(table_width*0.5-0.05,table_depth*0.5-0.05,table_height*0.5),world=table_world,mass=float('inf')))
table_surf.appearance().setColor(0.4,0.3,0.2)
for l in table_legs:
    l.appearance().setColor(0.4,0.3,0.2)

def make_object(clone_obj,world):
    obj = world.makeRigidObject("object"+str(world.numRigidObjects()))
    obj.geometry().set(clone_obj.geometry())
    obj.appearance().set(clone_obj.appearance())
    obj.setMass(clone_obj.getMass())
    return obj

#read the docs for this method in ../common/stable_faces.py
obj_stable_faces = stable_faces(obj,stability_tol=-0.001,merge_tol=0.05)

########################## Problem 2.A code goes in this cell ##############################

def sample_arrangement(k):
    global table_width,table_depth,table_height,table_world,obj
    for i in range(k):
        objsample = make_object(obj,table_world)
        R = so3.identity()
        t = [0,0,0]
        #TODO: do something to sample a rotation and translation -- use obj_stable_faces.
        #Feel free to make your own subroutines?
        objsample.setTransform(R,t)

##########################################################################################

sample_arrangement(5)

vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",table_world)
vis.show()

B. Fair enough, but this function doesn't generate "plausible" worlds since the objects are very likely to overlap with one another.  Using a set of at least 5 object types (e.g., the object filenames given below), create a function `sample_table_arrangement` that samples uniformly from the object set, and samples collision-free poses for each object.  The selected objects and their poses should be randomized every time the below cell is run.

In [ ]:
########################## Problem 2.B code goes in this cell ##############################

def sample_table_arrangement(object_set,k,max_iters=100):
    """Samples collision-free a table arrangement of k objects
    out of the given object set.
    
    Use the global variables 
    
    Args:
        object_set (list of str): some set of possible file names
        k (int) number of objects to instantiate
        max_iters (int) iterations.
    
    Returns:
        int: the number of successfully placed items
    """
    global table_width,table_depth,table_height,table_world
    #TODO: your code here
    sample_arrangement(k)
    return 0

##########################################################################################


In [ ]:
for i in reversed(range(table_world.numRigidObjects())):
    table_world.remove(table_world.rigidObject(i))

obj_fns = ["../data/objects/ycb-select/002_master_chef_can/nontextured.ply",
           "../data/objects/ycb-select/003_cracker_box/nontextured.ply",
           "../data/objects/ycb-select/011_banana/nontextured.ply",
           "../data/objects/ycb-select/021_bleach_cleanser/nontextured.ply",
           "../data/objects/ycb-select/048_hammer/nontextured.ply"]
feas = sample_table_arrangement(obj_fns,5)
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.addText("feas","%d placements feasible?"%feas,position=(5,5))
vis.add("world",table_world)
vis.show()

## Problem 3: Grasp Planning for Antipodal Grasps

Problem 3 asks to find antipodal grasps for a given object geometry, that is, line segments that a gripper could plausibly use for closing onto the geometry.  Here we will use the `AntipodalGrasp` data structure to represent these grasps, as well as the `GripperInfo` data structure. Read the GripperInfo documentation to better understand the meaning of these members.

The key defining parameters of an antipodal grasp include the center of the line segment and its axis. From these parameters, you should be able to fill in its contact points and the grasp width using some geometric computations. 

A. First, you will fill in the `fill_in_grasp` function, which takes a center and axis and fills out the other members of the AntipodalGrasp data structure.  The [Geometry3D.rayCast_ext](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html#klampt.Geometry3D.rayCast_ext) method used in `ray_cast_to_contact_point` will be helpful for you to retrieve the normal and position of a contact point. The subsequent cell will show the resulting grasps in red if they have no contact point, and green if they do have a contact point.  Moreover, the endpoints of the grasp will be arranged slightly outside of the object.

In [ ]:
#need to define this... will come later
object_normals = None

def grasp_from_contacts(contact1,contact2):
    """Helper: if you have two contacts, this returns an AntipodalGrasp"""
    d = vectorops.unit(vectorops.sub(contact2.x,contact1.x))
    grasp = AntipodalGrasp(vectorops.interpolate(contact1.x,contact2.x,0.5),d)
    grasp.finger_width = vectorops.distance(contact1.x,contact2.x)
    grasp.contact1 = contact1
    grasp.contact2 = contact2
    return grasp

def ray_cast_to_contact_point(obj,source,direction):
    """Produces a ContactPoint for the point hit by the ray:
        x(t) = source + t*direction
    
    Assumes object_normals is set up to match the object.
    
    Arguments:
        obj (RigidObjectModel)
        source (3-vector)
        direction (3-vector)
    
    Returns: ContactPoint with `x` set to the hit point,
    `n` set to the triangle normal, and coefficient of friction = 1.
    
    None is returned if the object is not hit by the ray.
    """
    assert object_normals is not None
    hit_tri,pt = obj.geometry().rayCast_ext(source,direction)
    if hit_tri < 0:
        return None
    return ContactPoint(pt,object_normals[hit_tri],1)

def fill_in_grasp(grasp,rigid_object):
    """TODO: you should fill this out to generate
    grasp.contact1, grasp.contact2 and grasp.finger_width.  
    
    Arguments:
        grasp (AntipodalGrasp): a partially-specified grasp (center and axis)
        rigid_object (RigidObjectModel): the object
    
    Returns None. (Just fill in the members of the grasp object.)
    """
    global object_normals
    if grasp.contact1 is not None:
        #assume already filled out
        return
    #put your code here
    return
    
###########################################################################################

In [ ]:
#Problem 3.A. Basic test of the fill-in method
from klampt.model import geometry

vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)

grasp1 = AntipodalGrasp([0,0,0.02],[1,0,0])
grasp2 = AntipodalGrasp([0,0.2,0.02],[1,0,0])
grasp3 = AntipodalGrasp([0,0,0.02],[0,1,0])
grasps = [grasp1,grasp2,grasp3]
object_normals = geometry.triangle_normals(obj.geometry())
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    fill_in_grasp(g,obj)
    if g.contact1 is None:
        g.add_to_vis(name,(1,0,0,1)) 
    else:
        g.add_to_vis(name,(0,1,0,1)) 
vis.show()

B. Next, you will implement a grasp scoring technique.  Fill in the components of the `score` method to generate a scoring function for each antipodal grasp.  Your scoring function should implement at least 3 of the `within_fingers`, `centering_alignment`, `normal_alignment`, `off_axis_torque`, and `finger_clearance` features as described in their documentation.  The final score should weight between them. 

Read the documentation of the `GripperInfo` data structure to better understand the characteristics of the `self.gripper_info` object, which declares some of the key geometric parameters of the gripper. In particular, `maximum_span`, `finger_length`, `finger_width`, and `finger_depth` will be useful here.

In the second cell below, you will be given a good grasp and two bad grasps. Grasps that are obviously good should show up as green, and grasps that are obviously bad should show up as red.

In [ ]:
########################## Problem 3.B code goes in this cell ##############################

class GraspScore:
    """
    Your class for scoring grasps.
    
    Attributes:
        gripper_info (GripperInfo): see common/gripper_info.py
        rigid_object (RigidObjectModel):
        position_uncertainty (float): a value used to represent the
            amount of uncertainty we have in the gripper positioning.
        gravity (3-tuple): the gravity vector.
    """
    def __init__(self,gripper_info,rigid_object):
        self.gripper_info = gripper_info
        self.rigid_object = rigid_object
        global object_normals
        object_normals = geometry.triangle_normals(rigid_object.geometry())
        self.position_uncertainty = 0.02
        self.gravity = (0,0,-9.8)

    def within_fingers(self,grasp):
        """Returns 1 if the object fits within gripper_info.maximum_span-position_uncertainty
        units along the specified grasp axis, and 0 if it has width greater than 
        gripper_info.maximum_span width, and otherwise decaying linearly from 1 to 0.

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        return 1

    def centering_alignment(self,grasp):
        """Returns the absolute error between the grasp center and the midpoint 
        between the points touched on the object.  If c is the center and
        m is the midpoint between the contact points p1 and p2, then this
        returns ||c-m||.

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        return 0

    def normal_alignment(self,grasp):
        """Returns the total angular error between the grasp axis and the normals
        of the object.  Specifically, returns angle(axis,n1) + angle(axis,n2) in
        radians, with n1 and n2 being the normals of the contact points.  (Choose
        the sign of axis that minimizes the angle.)

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        return 0

    def off_axis_torque(self,grasp):
        """Returns the magnitude of the torque imposed by gravity on the object
        for the given AntipodalGrasp.  (Don't forget to include the object's mass
        in your calculation).
        """
        #TODO: your code here
        return 0

    def finger_clearance(self,grasp):
        """Returns how much clearance the fingers have about the grasp.

        Imagine a point finger_radius away from the contact points, this
        should return min(d/gripper_info.finger_radius,1) where d is the minimum
        distance between the point and surrounding obstacles.
        """
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        if grasp.contact1 is None: return 0
        return 1

    def score(self,grasp):
        """Returns a score indicating a quality of the grasp. Lower is better.
        """
        within = self.within_fingers(grasp)
        center = self.centering_alignment(grasp)
        normal = self.normal_alignment(grasp)
        off_axis = self.off_axis_torque(grasp)
        clearance = self.finger_clearance(grasp)
        #print(within,center,normal,off_axis,clearance)
        if within == 0: return float('inf')
        #TODO: weight between these components
        return (1.0-within) + center + normal + off_axis + (1.0-clearance)

###########################################################################################

In [ ]:
#Problem 3.B. Basic test of the quality of one obviously good grasp and two obviously bad ones.
#Feel free to debug your code by testing new grasps here
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)

grasp1 = AntipodalGrasp([0,0,0.02],[1,0,0])
grasp2 = AntipodalGrasp([0,0.2,0.02],[1,0,0])
grasp3 = AntipodalGrasp([0,0,0.02],[0,1,0])
score = GraspScore(known_grippers.robotiq_140,obj)
grasps = [(g,score.score(g)) for g in [grasp1,grasp2,grasp3]]
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
vis.show()

C. Define a volume-centric grasp selection strategy in `antipodal_grasp_sample_volume` that samples grasps by sampling a random center point within the object's volume, and then samples a random direction for the axis.  From these N grasps, it should score and choose the top k grasps. The third cell below will show the results of your grasp sampler on whatever object was loaded in the initial cell at the top of this workbook.  By changing the object (try the banana and hammer!), you should observe different behavior.

In [ ]:
########################## Problem 3.C code goes in this cell ##############################

def antipodal_grasp_sample_volume(gripper,rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled centers and axes within the object
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    scorer = GraspScore(gripper,rigid_object)
    #this is a bounding box for the object
    bmin,bmax = rigid_object.geometry().getBBTight()
    grasps = []
    #TODO... add AntipodalGrasps to the grasps list
    scores = [(scorer.score(g),g) for g in grasps]
    scores = sorted(scores,key = lambda x:x[0])
    if k > len(grasps): k=len(grasps)
    return [(x[1],x[0]) for x in scores[:k]]

#############################################################################

In [ ]:
#Problem 3.C. Run your volume-based grasp sampler.
vis.createWindow()
vis.setViewport(closeup_viewport)
#uncomment this if you want to try your hand at point cloud grasp generation
"""
if world.numRigidObjects() == 1:
    obj_pc = world.makeRigidObject("object1_pc")
    obj_pc.geometry().set(obj.geometry().convert('PointCloud',0.02))
    translation = (0.3,0,0)
    m = obj.getMass()
    m.setCom(vectorops.add(m.getCom(),translation))
    obj_pc.setMass(m)
    obj_pc.geometry().translate(translation)
    obj_pc.appearance().setColor(1,0,1,0.5)
    obj = world.rigidObject(0)
"""
vis.add("world",world)

t0 = time.time()
grasps = antipodal_grasp_sample_volume(known_grippers.robotiq_140,obj,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 

#uncomment this if you want to try your hand at point cloud grasp generation
"""
t0 = time.time()
grasps = antipodal_grasp_sample_volume(known_grippers.robotiq_140,obj_pc,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "pc_grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
"""
vis.show()D. **(4-credit section only)**

Define a surface-centric grasp selection strategy in `antipodal_grasp_sample_surface` that samples grasps by sampling points on the object at random.  The approach should generate a list of candidates by sampling two surface points at random, and then construct an AntipodalGrasp through the points.  As before, it should then select the top `k` scoring grasps.


D. **(4-credit section only)**

Define a surface-centric grasp selection strategy in `antipodal_grasp_sample_surface` that samples grasps by sampling points on the object at random.  The approach should generate a list of candidates by sampling two surface points at random, and then construct an AntipodalGrasp through the points.  As before, it should then select the top `k` scoring grasps.


In [ ]:
########################## Problem 3.D code goes in this cell ##############################

def sample_trimesh(verts,tris,N):
    """Helper: samples N points on a triangle mesh by sampling a
    random triangle and a random point on the triangle.
    
    Returns: a pair (points,normals) of Nx3 arrays
    """
    global object_normals
    sample = [random.randint(0,len(verts)-1) for i in range(N)]
    pts = []
    normals = []
    for elem in sample:
        a,b,c = tris[elem]
        va = verts[a]
        vb = verts[b]
        vc = verts[c]
        u = random.random()
        v = random.random()
        if u+v > 1:
            v2 = (1-u)
            u2 = (1-v)
            u,v = u2,v2
        pt = va*(1-u-v) + vb*u + vc*v
        n = object_normals[elem]
        pts.append(pt)
        normals.append(n)
    return pts,normals


def antipodal_grasp_sample_surface(gripper,rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled pairs of points on the surface of
    the object.
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    scorer = GraspScore(gripper,rigid_object)
    grasps = []
    #TODO... add AntipodalGrasps to the grasps list
    scores = [(scorer.score(g),g) for g in grasps]
    scores = sorted(scores,key = lambda x:x[0])
    if k > len(grasps): k=len(grasps)
    return [(x[1],x[0]) for x in scores[:k]]

#############################################################################

#add your testing code here


Which approach is more efficient at generating high-quality grasps: volume sampling or surface sampling?  Compare the quality of grasps generated by these techniques, according to your scoring function.  Try running them multiple times to see best-case, worst-case, and average-case quality.  Report how many runs you used, and interpret the results in the written portion at the end of this document.  (Don't worry if your tests are inconclusive -- the best sampler and grasp scores are very much open matters for debate!)

**Written answers for 3.D go here**